# EXERCICE 1

# BRAINSTORMING


### Facteurs d'impacts de circulation

In [32]:
import pandas as pd

facteurs = [
    {"facteur": "Temporalité (heure/jour/semaine/saison)", "importance": 10, "commentaire": "Effet structurel majeur"},
    {"facteur": "Événements circulation (routes barrées, alternats, etc.)", "importance": 9, "commentaire": "Impacts ponctuels forts"},
    {"facteur": "Travaux / chantiers", "importance": 8, "commentaire": "Réduction capacité, déviations"},
    {"facteur": "Météo (pluie, vent, visibilité)", "importance": 7, "commentaire": "Vitesse/risk"},
    {"facteur": "Vacances scolaires", "importance": 6, "commentaire": "Modifie les flux domicile-travail"},
    {"facteur": "Jours fériés / ponts", "importance": 6, "commentaire": "Déplacements atypiques"},
    {"facteur": "Offre/perturbations transports (TBM)", "importance": 5, "commentaire": "Report modal"},
    {"facteur": "Grands événements (stade, salons, concerts)", "importance": 5, "commentaire": "Pics localisés"},
]

df_facteurs = pd.DataFrame(facteurs).sort_values("importance", ascending=False).reset_index(drop=True)
df_facteurs


,facteur,importance,commentaire
0,Temporalité (heure/jour/semaine/saison),10,Effet structurel majeur
1,"Événements circulation (routes barrées, altern...",9,Impacts ponctuels forts
2,Travaux / chantiers,8,"Réduction capacité, déviations"
3,"Météo (pluie, vent, visibilité)",7,Vitesse/risk
4,Vacances scolaires,6,Modifie les flux domicile-travail
5,Jours fériés / ponts,6,Déplacements atypiques
6,Offre/perturbations transports (TBM),5,Report modal
7,"Grands événements (stade, salons, concerts)",5,Pics localisés


In [33]:
df_facteurs.to_csv("perimetre_facteurs.csv", index=False, encoding="utf-8")
print("OK - perimetre_facteurs.csv créé")


OK - perimetre_facteurs.csv créé


# Table des sources


In [34]:
sources = [
    {
        "cible_ou_facteur": "Trafic (cible à prédire)",
        "source": "Bordeaux Métropole - ci_trafi_l",
        "type_acces": "API Opendatasoft (records/exports)",
        "travail_necessaire": "Historiser (snapshots), filtrer rocade, aligner timestamps",
    },
    {
        "cible_ou_facteur": "Événements circulation",
        "source": "Bordeaux Métropole - ci_evenmt_p",
        "type_acces": "API Opendatasoft (records)",
        "travail_necessaire": "Transformer en variable 'actif' sur période + rattachement spatial",
    },
    {
        "cible_ou_facteur": "Vacances scolaires",
        "source": "Ministère Éducation - fr-en-calendrier-scolaire",
        "type_acces": "API Opendatasoft (records) / data.gouv",
        "travail_necessaire": "Filtrer Bordeaux/zone, générer variable binaire vacances",
    },
    {
        "cible_ou_facteur": "Jours fériés",
        "source": "calendrier.api.gouv.fr / dataset data.gouv",
        "type_acces": "JSON (API)",
        "travail_necessaire": "Créer variable binaire férié + ponts (règles)",
    },
    {
        "cible_ou_facteur": "Météo",
        "source": "SYNOP (Météo-France) via data.gouv",
        "type_acces": "CSV/flux selon source",
        "travail_necessaire": "Choisir station (ex Mérignac), resampler et aligner au pas temps",
    },
    {
        "cible_ou_facteur": "Transports TBM",
        "source": "TBM GTFS (transport.data.gouv.fr)",
        "type_acces": "Téléchargement GTFS (zip)",
        "travail_necessaire": "Parser GTFS, construire indicateurs (offre/perturbations si RT)",
    },
]

df_sources = pd.DataFrame(sources)
df_sources


,cible_ou_facteur,source,type_acces,travail_necessaire
0,Trafic (cible à prédire),Bordeaux Métropole - ci_trafi_l,API Opendatasoft (records/exports),"Historiser (snapshots), filtrer rocade, aligne..."
1,Événements circulation,Bordeaux Métropole - ci_evenmt_p,API Opendatasoft (records),Transformer en variable 'actif' sur période + ...
2,Vacances scolaires,Ministère Éducation - fr-en-calendrier-scolaire,API Opendatasoft (records) / data.gouv,"Filtrer Bordeaux/zone, générer variable binair..."
3,Jours fériés,calendrier.api.gouv.fr / dataset data.gouv,JSON (API),Créer variable binaire férié + ponts (règles)
4,Météo,SYNOP (Météo-France) via data.gouv,CSV/flux selon source,"Choisir station (ex Mérignac), resampler et al..."
5,Transports TBM,TBM GTFS (transport.data.gouv.fr),Téléchargement GTFS (zip),"Parser GTFS, construire indicateurs (offre/per..."


# Racine projet + dossiers datalake

In [35]:
from pathlib import Path

RACINE_PROJET = Path(r"C:\Users\HP-X360-1030-G3\PycharmProjects\notebooks\exercice1")

DATA = RACINE_PROJET / "data"
RAW = DATA / "raw"
PARQUET = DATA / "parquet"

RAW.mkdir(parents=True, exist_ok=True)
PARQUET.mkdir(parents=True, exist_ok=True)

print("OK - Racine projet :", RACINE_PROJET)
print("OK - RAW :", RAW)
print("OK - PARQUET :", PARQUET)


OK - Racine projet : C:\Users\HP-X360-1030-G3\PycharmProjects\notebooks\exercice1
OK - RAW : C:\Users\HP-X360-1030-G3\PycharmProjects\notebooks\exercice1\data\raw
OK - PARQUET : C:\Users\HP-X360-1030-G3\PycharmProjects\notebooks\exercice1\data\parquet


# import dependances

In [36]:
import sys
!{sys.executable} -m pip install --upgrade pip


In [37]:
import requests
import pandas as pd
from datetime import datetime
import uuid
from typing import Any
from pathlib import Path


## API Opendatasoft + écriture Parquet datée


### horodatage + nom de fichier


In [38]:
def horodatage() -> str:
    return datetime.now().strftime("%Y-%m-%d_%H%M%S")

def nom_parquet(source: str) -> str:
    return f"{source}_{horodatage()}_{uuid.uuid4().hex[:8]}.parquet"


### sauvegarde parquet

In [39]:
def sauver_parquet(df: pd.DataFrame, source: str) -> Path:
    dossier = PARQUET / source
    dossier.mkdir(parents=True, exist_ok=True)

    chemin = dossier / nom_parquet(source)
    df.to_parquet(chemin, index=False)

    return chemin


### lecture OPENDATASOFT

In [40]:
def lire_opendatasoft(
    portail: str,
    dataset_id: str,
    limit_page: int = 100,
    nb_max: int | None = None,
    where: str | None = None,
    refinements: list[str] | None = None
) -> pd.DataFrame:
    url = f"{portail.rstrip('/')}/api/explore/v2.1/catalog/datasets/{dataset_id}/records"

    offset = 0
    lignes: list[dict[str, Any]] = []

    while True:
        params: list[tuple[str, str]] = [("limit", str(limit_page)), ("offset", str(offset))]

        if where:
            params.append(("where", where))

        if refinements:
            for r in refinements:
                params.append(("refine", r))

        r = requests.get(url, params=params, timeout=60)
        r.raise_for_status()

        data = r.json()
        results = data.get("results", [])

        if not results:
            break

        lignes.extend(results)

        if nb_max is not None and len(lignes) >= nb_max:
            lignes = lignes[:nb_max]
            break

        if len(results) < limit_page:
            break

        offset += limit_page

    if not lignes:
        return pd.DataFrame()

    return pd.json_normalize(lignes)


### lecture jours feries

In [41]:
def lire_jours_feries_metropole() -> pd.DataFrame:
    url = "https://calendrier.api.gouv.fr/jours-feries/metropole.json"
    r = requests.get(url, timeout=60)
    r.raise_for_status()

    d = r.json()

    df = pd.DataFrame([{"date": k, "nom": v} for k, v in d.items()])
    df["date"] = pd.to_datetime(df["date"], errors="coerce")

    return df.sort_values("date").reset_index(drop=True)


### Test rapide

In [42]:
#  jours fériés
df_feries = lire_jours_feries_metropole()
df_feries.head()

#  trafic (limité à 200 lignes pour tester)
df_test = lire_opendatasoft("https://datahub.bordeaux-metropole.fr", "ci_trafi_l", nb_max=200)
print("lignes test =", len(df_test))
df_test.head(3)


lignes test = 200


,gml_id,gid,ident,typevoie,etat,cdate,mdate,origine,commune,code_commune,geo_point_2d.lon,geo_point_2d.lat,geo_shape.type,geo_shape.geometry.coordinates,geo_shape.geometry.type
0,CI_TRAFI_L.2151,2151,I83,PENETRANTE,FLUIDE,2020-12-10T16:04:32+00:00,2025-12-18T10:40:50+00:00,PC_CIRCULATION,Bordeaux,33063,-0.601554,44.821867,Feature,"[[-0.601044, 44.82288], [-0.601087, 44.822671]...",LineString
1,CI_TRAFI_L.2152,2152,I60,PENETRANTE,FLUIDE,2020-12-10T16:05:56+00:00,2025-12-18T10:40:50+00:00,PC_CIRCULATION,Talence,33522,-0.600889,44.818853,Feature,"[[-0.59964, 44.818933], [-0.59992, 44.818933],...",LineString
2,CI_TRAFI_L.2153,2153,J05,PENETRANTE,FLUIDE,2020-12-10T16:59:01+00:00,2025-12-18T10:40:50+00:00,PC_CIRCULATION,Mérignac,33281,-0.621735,44.826486,Feature,"[[-0.624132, 44.826325], [-0.622647, 44.826484...",LineString


## Ingestion minimale

In [43]:
#  Trafic
df_trafic = lire_opendatasoft("https://datahub.bordeaux-metropole.fr", "ci_trafi_l")
print("trafic lignes =", len(df_trafic))
print("trafic ->", sauver_parquet(df_trafic, "ci_trafi_l"))

#  circulation
df_evenements = lire_opendatasoft("https://datahub.bordeaux-metropole.fr", "ci_evenmt_p")
print("evenements lignes =", len(df_evenements))
print("evenements ->", sauver_parquet(df_evenements, "ci_evenmt_p"))

#  Vacances scolaires
df_vacances = lire_opendatasoft(
    "https://data.education.gouv.fr",
    "fr-en-calendrier-scolaire",
    refinements=["location:Bordeaux"]
)
print("vacances lignes =", len(df_vacances))
print("vacances ->", sauver_parquet(df_vacances, "calendrier_scolaire"))

#  Jours fériés
df_feries = lire_jours_feries_metropole()
print("feries lignes =", len(df_feries))
print("feries ->", sauver_parquet(df_feries, "jours_feries_metropole"))


trafic lignes = 694
trafic -> C:\Users\HP-X360-1030-G3\PycharmProjects\notebooks\exercice1\data\parquet\ci_trafi_l\ci_trafi_l_2025-12-18_114254_8d33960e.parquet
evenements lignes = 12
evenements -> C:\Users\HP-X360-1030-G3\PycharmProjects\notebooks\exercice1\data\parquet\ci_evenmt_p\ci_evenmt_p_2025-12-18_114255_1168d5e7.parquet
vacances lignes = 68
vacances -> C:\Users\HP-X360-1030-G3\PycharmProjects\notebooks\exercice1\data\parquet\calendrier_scolaire\calendrier_scolaire_2025-12-18_114255_e613cc2f.parquet
feries lignes = 285
feries -> C:\Users\HP-X360-1030-G3\PycharmProjects\notebooks\exercice1\data\parquet\jours_feries_metropole\jours_feries_metropole_2025-12-18_114256_33048b8c.parquet
